### Using argopy to fetch argo index for 2019

In [1]:
from argopy import IndexFetcher as ArgoIndexFetcher
import pandas as pd
import numpy as np

index_loader = ArgoIndexFetcher()
df=index_loader.region([-180,180,-90.,90.,'2019-01-01','2019-12-31']).to_dataframe()
df.head()

/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


,file,date,longitude,latitude,ocean,date_update,wmo,institution,profiler_type
0,coriolis/6901832/profiles/R6901832_315.nc,2019-01-01 00:04:00,39.500,41.796,A,2019-01-01 00:30:02,6901832,"Ifremer, France","Arvor, Seabird conductivity sensor"
1,coriolis/6901726/profiles/R6901726_090.nc,2019-01-01 00:19:00,-44.372,46.497,A,2019-01-01 08:21:18,6901726,"Ifremer, France","Arvor, Seabird conductivity sensor"
2,jma/5905846/profiles/R5905846_001.nc,2019-01-01 00:30:01,-130.138,-9.963,P,2019-11-22 06:55:03,5905846,"JMA, Japan",Ninja-D deep float
3,aoml/5905759/profiles/D5905759_145D.nc,2019-01-01 00:31:15,-170.105,-28.816,P,2020-02-21 22:44:01,5905759,"AOML, USA","Solo, Seabird conductivity sensor"
4,aoml/1901639/profiles/R1901639_227.nc,2019-01-01 00:31:50,-21.594,-2.603,A,2019-01-10 20:00:38,1901639,"AOML, USA",S2A float


### Group by WMO so we get all floats, even dead ones

In [2]:
ldat=df.groupby('wmo').last()
ldat.head(5)                   

,file,date,longitude,latitude,ocean,date_update,institution,profiler_type
wmo,,,,,,,,
1900726,aoml/1900726/profiles/D1900726_410.nc,2019-04-01 14:38:35,-133.879,-24.811,P,2019-12-10 13:41:13,"AOML, USA","Webb Research, Seabird sensor"
1900816,aoml/1900816/profiles/R1900816_348.nc,2019-02-08 23:13:20,41.152,-28.603,I,2019-02-09 19:04:08,"AOML, USA","Webb Research, Seabird sensor"
1900973,aoml/1900973/profiles/D1900973_343.nc,2019-02-17 14:15:59,-167.086,-54.998,P,2019-02-28 14:24:58,"AOML, USA","Webb Research, Seabird sensor"
1900975,aoml/1900975/profiles/R1900975_349.nc,2019-04-22 21:42:33,142.011,-50.654,I,2019-04-24 01:04:17,"AOML, USA","Webb Research, Seabird sensor"
1900977,aoml/1900977/profiles/R1900977_361.nc,2019-08-29 10:50:29,-156.942,-53.043,P,2019-08-30 01:04:49,"AOML, USA","Webb Research, Seabird sensor"


### Build custom pd.Dataframes for plotting

In [3]:
#Reorder institutions
instit=['AOML, USA', 'MEDS, Canada', 'Ifremer, France', 'BODC, United Kingdom',
        'CSIRO, Australia', 'CSIO China Second Institute of Oceanography',
        'INCOIS, India', 'JMA, Japan', 'KMA, Korea','KORDI, Korea'] 
# For future plot
instit2=['AOML','MEDS','IFREMER','BODC','CSIRO','CSIO','INCOIS','JAMSTEC','KMA','KORDI']
count_a=np.zeros_like(instit,dtype=float)
count_i=np.zeros_like(instit,dtype=float)
count_p=np.zeros_like(instit,dtype=float)
i=0

for inst in instit:
    si=ldat.query('institution == "'+inst+'"').groupby(['ocean']).size()
    for j in range(len(si)):
        if si.index[j]=="A":
            count_a[i] = si['A'];
        elif si.index[j]=="P":
            count_p[i] = si['P'];
        elif si.index[j]=="I":
            count_i[i] = si['I'];
    i+=1
    
d = {'institution': instit2, 'Atlantic': count_a, 'Pacific': count_p, 'Indian': count_i, 'Continent': ['America','America','Europe','Europe','Australia','Asia','Asia','Asia','Asia','Asia']}
df = pd.DataFrame(data=d)

df

,institution,Atlantic,Pacific,Indian,Continent
0,AOML,489.0,1658.0,369.0,America
1,MEDS,58.0,64.0,0.0,America
2,IFREMER,638.0,85.0,70.0,Europe
3,BODC,196.0,6.0,66.0,Europe
4,CSIRO,13.0,178.0,198.0,Australia
5,CSIO,0.0,64.0,43.0,Asia
6,INCOIS,3.0,10.0,144.0,Asia
7,JAMSTEC,2.0,242.0,31.0,Asia
8,KMA,0.0,49.0,0.0,Asia
9,KORDI,0.0,4.0,0.0,Asia


### Using bokeh lib to make beautiful plot

In [4]:
from bokeh.plotting import figure, show
from bokeh.models import Label
from collections import OrderedDict
from bokeh.io import output_notebook

ocean_color = OrderedDict([("Atlantic","#428bca"),("Pacific", "#2e4045"),("Indian",  "#d9534f"  )])
continent_color = OrderedDict([("Asia", "#e69584"),("America", "#96ceb4"),("Europe", "#bae1ff"),("Australia", "#fff68f")])

width = 1000
height = 1000
inner_radius = 90
outer_radius = 300

minr = np.log10(1.)
maxr = np.log10(2000)
a = (outer_radius - inner_radius) / (maxr - minr)
b = outer_radius - a * maxr

def rad(mic):
    return a * np.log10(mic) + b

big_angle = 2.0 * np.pi / (len(df) + 1)
small_angle = big_angle / 7

p = figure(plot_width=width, plot_height=height, title="", x_axis_type=None, y_axis_type=None, 
           x_range=(-420, 420), y_range=(-420, 420),min_border=0, outline_line_color="black",
           background_fill_color="#f1f1f1")

p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None

# annular wedges
angles = np.pi/2 - big_angle/2 - df.index.to_series()*big_angle
colors = [continent_color[Continent] for Continent in df.Continent]
p.annular_wedge(0, 0, inner_radius, outer_radius, -big_angle+angles, angles, color=colors)

# small wedges
p.annular_wedge(0, 0, inner_radius, rad(df.Atlantic),-big_angle+angles+5*small_angle,
                -big_angle+angles+6*small_angle,color=ocean_color['Atlantic'])
p.annular_wedge(0, 0, inner_radius, rad(df.Pacific),-big_angle+angles+3*small_angle, 
                -big_angle+angles+4*small_angle, color=ocean_color['Pacific'])
p.annular_wedge(0, 0, inner_radius, rad(df.Indian), -big_angle+angles+1*small_angle,
                -big_angle+angles+2*small_angle, color=ocean_color['Indian'])

# circular axes and lables
labels = [1,10,100,1000]
radii = rad(labels)
p.circle(0, 0, radius=radii, fill_color=None, line_color="white")
p.text(0, radii, [str(r) for r in labels], text_font_size="10pt", text_align="center", text_baseline="middle")

# radial axes
p.annular_wedge(0, 0, inner_radius-5, outer_radius+5, -big_angle+angles, -big_angle+angles, color="black")

# DAC labels
xr = (radii[-1]+50)*np.cos(np.array(-big_angle/2 + angles))
yr = (radii[-1]+50)*np.sin(np.array(-big_angle/2 + angles))
label_angle=np.array(-big_angle/2+angles)
label_angle[label_angle < -np.pi/2] += np.pi # easier to read labels on the left side
p.text(xr, yr, df.institution, angle=label_angle, text_font_size="9pt", text_align="center", text_baseline="middle")

# OK, these hand drawn legends are pretty clunky, will be improved in future release
p.circle([-60, -60, 30, 30], [-360, -380, -360, -380], color=list(continent_color.values()), radius=7,line_color='black')
p.text([-45, -45, 45, 45], [-360, -380, -360, -380], text=[gr for gr in continent_color.keys()],
       text_font_size="10pt", text_align="left", text_baseline="middle")
p.rect([-30, -30, -30], [18, 0, -18], width=30, height=13, color=list(ocean_color.values()))
p.text([-5, -5, -5], [18, 0, -18], text=list(ocean_color), text_font_size="10pt", text_align="left", text_baseline="middle")

#TITLE
mytext = Label(x=0, y=350, text='Number of floats deployed per DAC & Ocean in 2019',text_font_size="12pt", text_align="center", text_baseline="middle")
p.add_layout(mytext)

output_notebook()

show(p)

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/series.py:679: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


Loading BokehJS ...